# Objectives

In this notebook I code various objective functions, and explain why each function is interesting. I also include a random search algorithm to test on each function.

In [1]:
import random as r
import numpy as np

## Random Search

In [2]:
class RandomSearch:
    def __init__(
        self,
        ranges
    ):
        self.ranges = ranges
        
    def generateRandomHypothesis(self):
        output = []
        for x in self.ranges:
            output.append(r.uniform(x[0], x[1]))
        return(output)
    
    def optimize(self,
                objectiveFunction,
                numIterations = 25000):
        bestX = None
        bestY = -float("inf")
        for i in range(numIterations):
            newX = self.generateRandomHypothesis()
            newY = objectiveFunction(newX)
            if newY > bestY:
                bestX = newX
                bestY = newY
        return(bestX)

## Physics Problems

### Drag Minimization 

The drag per second a vehicle travelling on a road experiences is approximately R(v) = Av^2 + Bv + C where A is a factor relating aerodynamics, B to rolling and drivetrain losses, and C is a constant load. If t is the time taken to arrive at a destination D, minimizing t x R will minimize the losses due to drag for the journey. Since v = D/t and D is constant this is equivalent to minimizing R/v, i.e. Av + B + C/v. 

Suppose we have a car with A = 0.6, B = 5, C = 80, we have a well-defined optimization problem. It's fun to play around with A/B/C and see how different physical parameters affect the optimum speed. most cars travel ~30mph most of the time, so we expect an answer near 13m/s

In [3]:
def dragMinimization(
    hypothesis,
    aerodynamicsConstant = 0.6,
    rollingConstant = 5,
    loadConstant = 80
):
    v = hypothesis[0]
    aTerm = aerodynamicsConstant * v 
    bTerm = rollingConstant
    cTerm = loadConstant / v
    output = - (aTerm + bTerm + cTerm)
    return(output)

In [4]:
rs = RandomSearch([[0, 50]])
print(rs.optimize(dragMinimization))

[11.544763078741893]


### Semi-Empirical Masss Formula (SEMF)

Below is the semi-empirical mass formula. It's a formula from physics which gives us the binding energy of a nucleus given its proton number, Z, and neutron number, N. Since physics is effectively trying to maximise the binding energy per nucleon, we have a fun optimization problem if we divide the energy predicted by the SEMF by Z + N. You can play around with physics and see which isotopes would be stable if you changed, say, the Coulomb term! 

SEMF is also useful since it has a known best result of Nickel-62: Z = 28, N = 34. The closer the output is to this, the better our algorithm is doing. Another very stable isotope is Iron-56: Z = 26, N = 30, so if we get a result close to this the algorithm has done well, too.

More about the SEMF here: https://en.wikipedia.org/wiki/Semi-empirical_mass_formula

In [5]:
def SEMF(
    hypothesis, 
    volumeConstant = 15.8,
    surfaceConstant = 17.8,
    coulombConstant = 0.711,
    asymmetryConstant = 23.7,
    pairingConstant = 11.18
):
    
    #Physical info from hypothesis
    Z = int(hypothesis[0]) #Proton number
    N = int(hypothesis[1]) #Neutron number
    A = Z + N              #Nucleon number
    
    #Calculate each term
    volumeTerm = volumeConstant * A
    surfaceTerm = -surfaceConstant * A ** (2/3)
    coulombTerm = -coulombConstant * Z * (Z - 1) * A ** (-1/3)
    asymmetryTerm = -asymmetryConstant * ((N - Z) ** 2) / A
    
    #Pairing term
    if A % 2 == 1:
        pairingTerm = 0
    elif Z % 2 == 0 and N % 2 == 0:
        pairingTerm = pairingConstant * A ** (-1/2)
    elif Z % 2 == 1 and N % 2 == 1:
        pairingTerm = -pairingConstant * A ** (-1/2)
    else:
        pairingTerm = 0
    output = (volumeTerm + surfaceTerm + coulombTerm + asymmetryTerm + pairingTerm) / A
    return(output)

rs = RandomSearch([[1, 120], [1, 180]])
print(rs.optimize(SEMF))

[26.928342728405557, 32.47247324528793]


## Game Theory Problems

### El Farol Bar

With El Farol Bar, we assume there are several bars in town, say 5. Each bar has a base "niceness" value N, and we're antisocial nerds so we prefer bars with fewer people in. Thus the utility we would get for attending each bar is N^-kx where x is the number of other people in the bar and k some decay constant. 

Classic El Farol asks us to come up with a strategy for which bar to go to given other people are more likely to go to nicer bars, but I prefer to ask what the optimal distribution of people across bars is to maximise their culmulative utility. Thus we maximise the sum of xN^-kx for each N, x. (For convenience, k is constant across all bars).

We enforce that the number of people in each bar must be an integer, and the total number of people in all bars is free to vary (some people stay home and write code for GitHub instead).

In [6]:
barUtilities = [10, 20, 40, 80, 160]
decayConstant = 0.5

def singleBarUtility(x, barNumber):
    output = x * barUtilities[barNumber] ** (-decayConstant * (x - 1))
    return(output)

def elFarol(hypothesis):
    peopleAssignments = [int(x) for x in hypothesis]
    output = 0
    for i in range(len(barUtilities)):
        output += singleBarUtility(peopleAssignments[i], i)
    return(output)

In [7]:
ranges = []
for i in range(len(barUtilities)):
    ranges.append([0, 50])

rs = RandomSearch(ranges)
distribution = rs.optimize(elFarol)
print([int(x) for x in distribution])

[3, 1, 8, 1, 1]


## Cousins Problem

Suppose you are in a room with 100 other people, playing a game with them. You pick a threshold N. If you are exactly Nth cousins with someone, you pay each other nothing. If you are more closely related than Nth cousins, they pay you £100/N. If you are more distantly related than Nth cousins, you pay them £100/N. What is the optimum N to pick to maximise your winnings? We assume cousins are normally distributed about mean X and standard deviation S.

In [ ]:
X = 15
S = 3

def generateCousins(numCousins = 100):
    output = [int(r.gauss(X, S)) for i in range(numCousins)]
    return(output)

cousinsList = generateCousins()

def cousins(x):
    N = int(x[0])
    output = sum((100 / N if N > C else (-100 / N if N < C else 0)) for C in cousinsList)
    return(output)

ranges = [1, 100]

rs = RandomSearch(ranges)
distribution = rs.optimize(cousins)


## Maths Problems

### Euler's Conjecture

Fermat's Last Theorem is that a^k + b^k = c^k has no integer solutions for k > 2. Euler's conjecture is a (now shown to be false) generalisation of this, which asserts that the sum of n terms like a^k has no positive integer solutions for k > n. 

A counterexample is:-

27^5 + 84^5 + 110^5 + 133^5 = 144^5

See: https://marktomforde.com/academic/miscellaneous/images/ShortestPaper.pdf

The below code constructs an objective function which returns 0 if the proposed solution is a valid counterexample to the Euler conjecture, and an increasingly negative value the further the proposed solution is from holding otherwise. If n >= k it simply returns -1,000,000.

It's an interesting objective because it breaks a lot of the assumptions that standard optimization makes. Suppose we have a solution which is only off from being a valid counterexample by 1, that isn't really any better than being off by 100. However, we need some way of quantifying how close a solution came to succesfully disproving Euler. Random search can't generally find good solutions, unless we run it for an extremely long time. So this objective function is a fun challenge for other optimization algorithms.

In [8]:
def EulerConjecture(hypothesis):
    k = int(hypothesis[0])
    b = int(hypothesis[1])
    if len(hypothesis) - 2 >= k:
        return(-(10 ** 6))
    else: 
        bases = [int(x) for x in hypothesis[2:]]
        output = -np.fabs(float(b ** k - np.sum([base ** k for base in bases])))
        if output == 0:
            print("Solution found: ", hypothesis)
        return(output)

#Demonstrating that the solution works
print(EulerConjecture([5, 144, 27, 84, 110, 133]))

#Allowing the length of the hypotheses to vary:
for h in range(10):
    ranges = [[1, 10]]
    for i in range(h + 5):
        ranges.append([0, 99])
    rs = RandomSearch(ranges)
    print("Closest answer found by random search for k =", h + 5, " is: ", rs.optimize(EulerConjecture))

Solution found:  [5, 144, 27, 84, 110, 133]
-0.0
Closest answer found by random search for k = 5  is:  [3.831751569333685, 97.26500136608789, 58.33628367366694, 35.05066673950456, 17.18454102515389, 49.38976568863983]
Closest answer found by random search for k = 6  is:  [5.173261289138336, 36.48363538712624, 60.48987320799617, 27.66571893155851, 31.066474669776465, 79.52042704274635, 94.58522426174038]
Closest answer found by random search for k = 7  is:  [6.184087967296084, 54.215850004673776, 80.2279272234784, 53.74549038359824, 86.8179915110833, 21.47614295838378, 41.41168333054684, 87.52046712098105]
Closest answer found by random search for k = 8  is:  [4.57906948403831, 16.342044275513356, 84.45314432617538, 15.51926362203062, 23.747280942405297, 73.97106162770125, 88.56076329422143, 44.74010343335052, 64.72637006549502]
Closest answer found by random search for k = 9  is:  [6.545931451676519, 33.04887837835249, 32.821844224805325, 4.78157135179562, 62.33916669457925, 80.3419404

## Polynomial Solver

Here you can input a polynomial in the form P = a + bx + cx^2 + dx^3 ... and the objective function rewards solutions which are closest to P = 0. In general polynomials of order N have at most N real solutions, and optimizing this way finds one of them at random. To find multiple, just run the alogirthm again until the others pop up. The example polynomial has solutions at -1. -2, and -3.

In [39]:
def polynomial(hypothesis, terms = [6, 11, 6, 1]):
    total = 0
    for i in range(len(terms)):
        total += terms[i] * (hypothesis[0] ** i)
    return(-np.fabs(total))

ranges = [[-10, 10]]
rs = RandomSearch(ranges)
rs.optimize(polynomial)

[-2.999742543935855]

## Engineering / Design

### Drinks Can

Suppose we want to design a cylindrical drinks can. The volume must be at least 330cm^3, but otherwise we want the smallest possible surface area. We have control over the radius R and the height H, and we find the can which optimizes these constraints. A real can I measured has R = 3.3cm and H = 12cm

In [53]:
def drinksCan(hypothesis):
    R = hypothesis[0]
    H = hypothesis[1]
    A = (2 * np.pi * (R ** 2)) + (2 * np.pi * R * H)
    V = (np.pi * (R ** 2)) * H
    volumeTerm = min(V - 330, 0)
    return(volumeTerm - A)

ranges = [[0, 20], [0, 20]]
rs = RandomSearch(ranges)
rs.optimize(drinksCan, 10 ** 6)

[3.6990209158576093, 7.675653903582928]